In [1]:
!pip install tensorflow_gpu==2.0.0-alpha0
##!pip install tensorflow_hub

     |████████████████████████████████| 332.1MB 62kB/s 
     |████████████████████████████████| 3.0MB 26.5MB/s 
     |████████████████████████████████| 61kB 24.0MB/s 
     |████████████████████████████████| 419kB 32.2MB/s 


In [2]:
!unzip src

Archive:  src.zip
   creating: src/.ipynb_checkpoints/
  inflating: src/.ipynb_checkpoints/utils-checkpoint.py  
   creating: src/__pycache__/
  inflating: src/__pycache__/utils.cpython-36.pyc  
   creating: src/data/
   creating: src/data/atis/
   creating: src/data/atis/test/
  inflating: src/data/atis/test/label  
  inflating: src/data/atis/test/seq.in  
  inflating: src/data/atis/test/seq.out  
   creating: src/data/atis/train/
  inflating: src/data/atis/train/label  
  inflating: src/data/atis/train/seq.in  
  inflating: src/data/atis/train/seq.out  
   creating: src/data/atis/valid/
  inflating: src/data/atis/valid/label  
  inflating: src/data/atis/valid/seq.in  
  inflating: src/data/atis/valid/seq.out  
   creating: src/data/snips/
   creating: src/data/snips/test/
  inflating: src/data/snips/test/label  
  inflating: src/data/snips/test/seq.in  
  inflating: src/data/snips/test/seq.out  
   creating: src/data/snips/train/
  inflating: src/data/snips/train/label  
  inflating:

In [0]:
import sys
sys.path.insert(0, '/content/src')
#sys.path.insert(0, './src')

In [0]:
import os
import argparse
import logging
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Layer, Conv1D,TimeDistributed, Bidirectional, Dense, Embedding
from tensorflow.compat.v1.keras.layers import CuDNNLSTM, LSTM
#import tensorflow_hub as hub
from utils import createVocabulary
from utils import loadVocabulary
from utils import computeF1Score
from utils import DataProcessor

In [0]:

parser = argparse.ArgumentParser(allow_abbrev=False)

#Network
parser.add_argument("--num_units", type=int, default=64, help="Network size.", dest='layer_size')
parser.add_argument("--model_type", type=str, default='full', help="""full(default) | intent_only
                                                                    full: full attention model
                                                                    intent_only: intent attention model""")

#Training Environment
parser.add_argument("--batch_size", type=int, default=48, help="Batch size.")
parser.add_argument("--max_epochs", type=int, default=50, help="Max epochs to train.")
parser.add_argument("--no_early_stop", action='store_false',dest='early_stop', help="Disable early stop, which is based on sentence level accuracy.")
parser.add_argument("--patience", type=int, default=20, help="Patience to wait before stop.")

#Model and Vocab
parser.add_argument("--dataset", type=str, default=None, help="""Type 'atis' or 'snips' to use dataset provided by us or enter what ever you named your own dataset.
                Note, if you don't want to use this part, enter --dataset=''. It can not be None""")
parser.add_argument("--model_path", type=str, default='./src/model', help="Path to save model.")
parser.add_argument("--vocab_path", type=str, default='./src/vocab', help="Path to vocabulary files.")

#Data
parser.add_argument("--train_data_path", type=str, default='train', help="Path to training data files.")
parser.add_argument("--test_data_path", type=str, default='test', help="Path to testing data files.")
parser.add_argument("--valid_data_path", type=str, default='valid', help="Path to validation data files.")
parser.add_argument("--input_file", type=str, default='seq.in', help="Input file name.")
parser.add_argument("--slot_file", type=str, default='seq.out', help="Slot file name.")
parser.add_argument("--intent_file", type=str, default='label', help="Intent file name.")

arg=parser.parse_args(["--dataset","snips"])

In [6]:
#Print arguments
for k,v in sorted(vars(arg).items()):
    print(k,'=',v)
print()

if arg.model_type == 'full':
    add_final_state_to_intent = True
    remove_slot_attn = False
elif arg.model_type == 'intent_only':
    add_final_state_to_intent = True
    remove_slot_attn = True
else:
    print('unknown model type!')
    exit(1)

#full path to data will be: ./data + dataset + train/test/valid
if arg.dataset == None:
    print('name of dataset can not be None')
    exit(1)
elif arg.dataset == 'snips':
    print('use snips dataset')
elif arg.dataset == 'atis':
    print('use atis dataset')
else:
    print('use own dataset: ',arg.dataset)
full_train_path = os.path.join('./src/data',arg.dataset,arg.train_data_path)
full_test_path = os.path.join('./src/data',arg.dataset,arg.test_data_path)
full_valid_path = os.path.join('./src/data',arg.dataset,arg.valid_data_path)

createVocabulary(os.path.join(full_train_path, arg.input_file), os.path.join(arg.vocab_path, 'in_vocab'))
createVocabulary(os.path.join(full_train_path, arg.slot_file), os.path.join(arg.vocab_path, 'slot_vocab'), flag='slot')
createVocabulary(os.path.join(full_train_path, arg.intent_file), os.path.join(arg.vocab_path, 'intent_vocab'),flag='int')

in_vocab = loadVocabulary(os.path.join(arg.vocab_path, 'in_vocab'))
slot_vocab = loadVocabulary(os.path.join(arg.vocab_path, 'slot_vocab'))
intent_vocab = loadVocabulary(os.path.join(arg.vocab_path, 'intent_vocab'))


batch_size = 48
dataset = snips
early_stop = True
input_file = seq.in
intent_file = label
layer_size = 64
max_epochs = 50
model_path = ./src/model
model_type = full
patience = 20
slot_file = seq.out
test_data_path = test
train_data_path = train
valid_data_path = valid
vocab_path = ./src/vocab

use snips dataset


In [0]:
class CustomDropout(Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs    

In [0]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units, kernel_regularizer = tf.keras.regularizers.l2(0.02))
        self.W2 = tf.keras.layers.Dense(units,kernel_regularizer = tf.keras.regularizers.l2(0.02))
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)


        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [0]:
#both query and values are time distributed
class CustomBahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(CustomBahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units,kernel_regularizer = tf.keras.regularizers.l2(0.02))
        #self.W2 = TimeDistributed(tf.keras.layers.Dense(units,kernel_regularizer = tf.keras.regularizers.l2(0.02)))
        self.W2 = Conv1D(units,5,1,'same')
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        #hidden_with_time_axis = tf.expand_dims(query, 1)
        hidden_with_time_axis = query

        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        #context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [0]:
class SlotGate(tf.keras.Model):
    def __init__(self, units):
        super(SlotGate, self).__init__()
        self.W1 = tf.keras.layers.Dense(units, kernel_regularizer = tf.keras.regularizers.l2(0.02))
        self.W2 = tf.keras.layers.Dense(units, kernel_regularizer = tf.keras.regularizers.l2(0.02))
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)
        
        # score shape == (batch_size, max_length, hidden_size)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        #context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [0]:
class MyModel(tf.keras.Model):
    def __init__(self, input_size, slot_size, intent_size, layer_size = 64):
        super(MyModel, self).__init__()
        self.attn_size = 2 * layer_size
        self.embedding = Embedding(input_size, layer_size)

        self.bilstm = Bidirectional(CuDNNLSTM(2*layer_size, return_sequences=True,
                                              return_state=True,kernel_regularizer = tf.keras.regularizers.l2(0.02) ))
        self.dropout = CustomDropout(0.3)
        self.slot_att = CustomBahdanauAttention(2*layer_size)
        self.intent_att = BahdanauAttention(2*layer_size)
        self.slot_gate = SlotGate(2*layer_size)
        #self.slot_proj = Conv1D(slot_size, 5, 1, 'same',activation='tanh')
        self.intent_proj = Dense(intent_size, activation='tanh')
        self.slot_proj = TimeDistributed(Dense(slot_size))
  
    @tf.function  
    def call(self, inputs, sequence_length, isTraining=True):
        x = self.embedding(inputs)
        state_outputs, fw_h, fw_c, bw_h, bw_c = self.bilstm(x) 
        state_outputs = self.dropout(state_outputs, isTraining)
        fwd_h = self.dropout(fw_h, isTraining)
        bw_h = self.dropout(bw_h, isTraining)
        fwd_c = self.dropout(fw_c, isTraining)
        bw_c = self.dropout(bw_c, isTraining)
        final_state = tf.keras.layers.concatenate([fw_h, fw_c, bw_h, bw_c], -1)

        slot_d, _ = self.slot_att(state_outputs, state_outputs)
        intent_d, _ = self.intent_att(final_state, state_outputs)

        #intent_out = tf.keras.layers.concatenate([intent_d,final_state], -1)
        #intent_out = final_state
        intent_out = intent_d

        slot_gated,_ = self.slot_gate(intent_out, slot_d)

        slot_out = tf.keras.layers.concatenate([slot_gated,state_outputs], -1)

        intent = self.intent_proj(intent_out)
        slots = self.slot_proj(slot_out)
        outputs = [slots, intent]
        return outputs

In [0]:

tf.keras.backend.clear_session()
model = MyModel(len(in_vocab['vocab']), len(slot_vocab['vocab']), len(intent_vocab['vocab']), layer_size=arg.layer_size)

In [0]:
def valid(in_path, slot_path, intent_path):
    data_processor_valid = DataProcessor(in_path, slot_path, intent_path, in_vocab, slot_vocab, intent_vocab)
    pred_intents = []
    correct_intents = []
    slot_outputs = []
    correct_slots = []
    input_words = []

    #used to gate
    #gate_seq = []
    while True:
        in_data, slot_data, slot_weight, length, intents, in_seq, slot_seq, intent_seq = data_processor_valid.get_batch(arg.batch_size)
        #feed_dict = {input_data.name: in_data, sequence_length.name: length}
        #ret = sess.run(inference_outputs, feed_dict)
        slots, intent = model(in_data, length, isTraining = False)
        for i in np.array(intent):
            pred_intents.append(np.argmax(i))
        for i in intents:
            correct_intents.append(i)

        pred_slots = slots
        for p, t, i, l in zip(pred_slots, slot_data, in_data, length):
            p = np.argmax(p, 1)
            tmp_pred = []
            tmp_correct = []
            tmp_input = []
            for j in range(l):
                tmp_pred.append(slot_vocab['rev'][p[j]])
                tmp_correct.append(slot_vocab['rev'][t[j]])
                tmp_input.append(in_vocab['rev'][i[j]])

            slot_outputs.append(tmp_pred)
            correct_slots.append(tmp_correct)
            input_words.append(tmp_input)

        if data_processor_valid.end == 1:
            break

    pred_intents = np.array(pred_intents)
    correct_intents = np.array(correct_intents)
    accuracy = (pred_intents==correct_intents)
    semantic_error = accuracy
    accuracy = accuracy.astype(float)
    accuracy = np.mean(accuracy)*100.0

    index = 0
    for t, p in zip(correct_slots, slot_outputs):
        # Process Semantic Error
        if len(t) != len(p):
            raise ValueError('Error!!')

        for j in range(len(t)):
            if p[j] != t[j]:
                semantic_error[index] = False
                break
        index += 1
    semantic_error = semantic_error.astype(float)
    semantic_error = np.mean(semantic_error)*100.0

    f1, precision, recall = computeF1Score(correct_slots, slot_outputs)
    print('slot f1: ' + str(f1) + '\tintent accuracy: ' + str(accuracy) + '\tsemantic_error: ' + str(semantic_error))
    #print('intent accuracy: ' + str(accuracy))
    #print('semantic error(intent, slots are all correct): ' + str(semantic_error))

    data_processor_valid.close()
    return f1,accuracy,semantic_error,pred_intents,correct_intents,slot_outputs,correct_slots,input_words#,gate_seq

In [0]:
#training
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
ckpt = tf.train.Checkpoint(step=tf.Variable(-1), optimizer=opt, net=model)
manager = tf.train.CheckpointManager(ckpt, arg.model_path + "/" + arg.dataset + '/tf_ckpts', max_to_keep=3)
data_processor = None
valid_err = 0
no_improve = 0
save_path = os.path.join(arg.model_path , str(arg.dataset) + "/")
for epoch in range(50):
    while True:
        if data_processor == None:
            i_loss = 0
            s_loss = 0
            batches = 0
            data_processor = DataProcessor(os.path.join(full_train_path, arg.input_file), os.path.join(full_train_path, arg.slot_file), os.path.join(full_train_path, arg.intent_file), in_vocab, slot_vocab, intent_vocab)
        in_data, slot_labels, slot_weights, length, intent_labels,in_seq,_,_ = data_processor.get_batch(arg.batch_size)
        
        with tf.GradientTape() as tape:
            slots, intent = model(in_data, length, isTraining = True)
            intent_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=intent_labels, logits=intent)  
            intent_loss = tf.reduce_sum(intent_loss)/tf.cast(arg.batch_size, tf.float32)
            slots_out = tf.reshape(slots, [-1,len(slot_vocab['vocab'])])
            slots_shape = tf.shape(slot_labels)
            slot_reshape = tf.reshape(slot_labels, [-1])
            crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=slot_reshape, logits=slots_out)
            crossent = tf.reshape(crossent, slots_shape)
            slot_loss = tf.reduce_sum(crossent*slot_weights, 1)
            total_size = tf.reduce_sum(slot_weights, 1)
            total_size += 1e-12
            slot_loss = slot_loss/ total_size
            slot_loss = tf.reduce_sum(slot_loss)
            
            total_loss = intent_loss + slot_loss + (tf.reduce_sum(model.losses)/50)    #intent_loss + 
        
        grads = tape.gradient(total_loss, model.trainable_weights)
        opt.apply_gradients(zip(grads, model.trainable_weights))
        s_loss = s_loss + tf.reduce_sum(slot_loss)/tf.cast(arg.batch_size, tf.float32)
        i_loss = i_loss +  tf.reduce_sum(intent_loss)/tf.cast(arg.batch_size, tf.float32)
        batches = batches + 1
        if data_processor.end == 1:
            data_processor.close()
            data_processor = None
            break
            
    #print("Training Epoch: " ,epoch," Slot Loss: ",s_loss/batches, " Intent_Loss: ", i_loss/batches)
    print("EPOCH: ", epoch, " *******************************************************************")
    print('Train:', end="\t")
    _ = valid(os.path.join(full_train_path, arg.input_file), os.path.join(full_train_path, arg.slot_file), os.path.join(full_train_path, arg.intent_file))
    
    print('Valid:', end="\t")
    epoch_valid_slot, epoch_valid_intent, epoch_valid_err,valid_pred_intent,valid_correct_intent,valid_pred_slot,valid_correct_slot,valid_words = valid(os.path.join(full_valid_path, arg.input_file), os.path.join(full_valid_path, arg.slot_file), os.path.join(full_valid_path, arg.intent_file))

    print('Test:', end="\t")
    epoch_test_slot, epoch_test_intent, epoch_test_err,test_pred_intent,test_correct_intent,test_pred_slot,test_correct_slot,test_words = valid(os.path.join(full_test_path, arg.input_file), os.path.join(full_test_path, arg.slot_file), os.path.join(full_test_path, arg.intent_file))
    
    ckpt.step.assign_add(1)
    if epoch_valid_err <= valid_err:
        no_improve += 1
    else:
        valid_err = epoch_valid_err
        no_improve = 0
        print("Saving", str(ckpt.step), "with valid accuracy:", valid_err   )
        save_path = manager.save()

    if arg.early_stop == True:
        if no_improve > arg.patience:
            print("EARLY BREAK")
            break

EPOCH:  0  *******************************************************************
Train:	slot f1: 49.30110087117286	intent accuracy: 70.9033934576582	semantic_error: 17.945582390706207
Valid:	slot f1: 47.49003984063745	intent accuracy: 68.57142857142857	semantic_error: 15.714285714285714
Test:	slot f1: 46.27181385510312	intent accuracy: 71.71428571428572	semantic_error: 17.57142857142857
Saving <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0> with valid accuracy: 15.714285714285714
EPOCH:  1  *******************************************************************
Train:	slot f1: 68.51097770874439	intent accuracy: 82.08498929990829	semantic_error: 40.47691837358606
Valid:	slot f1: 64.50250725785166	intent accuracy: 80.0	semantic_error: 34.85714285714286
Test:	slot f1: 63.80498145204028	intent accuracy: 81.85714285714286	semantic_error: 35.85714285714286
Saving <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1> with valid accuracy: 34.85714285714286
EPOCH:  2  ********************

In [15]:
 model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  719552    
_________________________________________________________________
bidirectional (Bidirectional multiple                  66560     
_________________________________________________________________
custom_dropout (CustomDropou multiple                  0         
_________________________________________________________________
custom_bahdanau_attention (C multiple                  98689     
_________________________________________________________________
bahdanau_attention (Bahdanau multiple                  49537     
_________________________________________________________________
slot_gate (SlotGate)         multiple                  33153     
_________________________________________________________________
dense_8 (Dense)              multiple                  269

In [16]:
#Let's try to clear slate and reload maodel  .....
import time
tf.keras.backend.clear_session()
if arg.dataset == 'atis':
    test_batch = 893
elif arg.dataset == 'snips':
    test_batch = 700

model = MyModel(len(in_vocab['vocab']), len(slot_vocab['vocab']), len(intent_vocab['vocab']), layer_size=arg.layer_size)
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

ckpt = tf.train.Checkpoint(step=tf.Variable(-1), optimizer=opt, net=model)
manager = tf.train.CheckpointManager(ckpt, arg.model_path + "/" + arg.dataset + '/tf_ckpts', max_to_keep=3)
ckpt.restore(manager.latest_checkpoint)

data_processor_test = DataProcessor(os.path.join(full_test_path, arg.input_file), os.path.join(full_test_path, arg.slot_file), os.path.join(full_test_path, arg.intent_file), in_vocab, slot_vocab, intent_vocab)
in_data, slot_labels, slot_weights, length, intent_labels,in_seq,_,_ = data_processor_test.get_batch(test_batch)
data_processor_test.close()



t = time.perf_counter()
slots, intent = model(in_data, length, isTraining = False)
elapsed = time.perf_counter() - t
print("Milli seconds per query:", (elapsed*1000)/float(100.0))

pred_intents = []
correct_intents = []
slot_outputs = []
correct_slots = []
input_words = []

for i in np.array(intent):
    pred_intents.append(np.argmax(i))
for i in intent_labels:
    correct_intents.append(i)

pred_slots = slots
for p, t, i, l in zip(pred_slots, slot_labels, in_data, length):
    p = np.argmax(p, 1)
    tmp_pred = []
    tmp_correct = []
    tmp_input = []
    for j in range(l):
        tmp_pred.append(slot_vocab['rev'][p[j]])
        tmp_correct.append(slot_vocab['rev'][t[j]])
        tmp_input.append(in_vocab['rev'][i[j]])

    slot_outputs.append(tmp_pred)
    correct_slots.append(tmp_correct)
    input_words.append(tmp_input)

pred_intents = np.array(pred_intents)
correct_intents = np.array(correct_intents)
accuracy = (pred_intents==correct_intents)
semantic_error = accuracy
accuracy = accuracy.astype(float)
accuracy = np.mean(accuracy)*100.0

index = 0
for t, p in zip(correct_slots, slot_outputs):
    # Process Semantic Error
    if len(t) != len(p):
        raise ValueError('Error!!')

    for j in range(len(t)):
        if p[j] != t[j]:
            semantic_error[index] = False
            break
    index += 1
semantic_error = semantic_error.astype(float)
semantic_error = np.mean(semantic_error)*100.0

f1, precision, recall = computeF1Score(correct_slots, slot_outputs)
print('slot f1: ' + str(f1) + '\tintent accuracy: ' + str(accuracy) + '\tsemantic_accuracy: ' + str(semantic_error))







W0609 19:14:56.006635 140374735677312 tf_logging.py:161] Entity <method-wrapper '__call__' of weakref object at 0x7faaca76d6d8> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env variable AUTOGRAPH_VERBOSITY >= 1. Please report this to the AutoGraph team. Cause: Object conversion is not yet supported. If you are trying to convert code that uses an existing object, try including the creation of that object in the conversion. For example, instead of converting the method of a class, try converting the entire class instead. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/README.md#using-the-functional-api for more information.
W0609 19:14:56.012698 140374735677312 tf_logging.py:161] Entity <method-wrapper '__call__' of weakref object at 0x7faaca7a48b8> could not be transformed and will be staged without change. Error details can be found in the logs when running with the env vari

Milli seconds per query: 33.75181055000212
slot f1: 83.16616969368391	intent accuracy: 70.28571428571428	semantic_accuracy: 45.857142857142854
